<a href="https://colab.research.google.com/github/Josepholaidepetro/GCP-Projects/blob/main/notebooks/Creating_a_sampled_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [ ]:
# Ensure the right version of Tensorflow is installed.
!pip freeze | grep tensorflow==2.1

In [ ]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-00-e0281c0549f5'
PROJECT = 'qwiklabs-gcp-00-e0281c0549f5'
REGION = 'us-central1'

In [ ]:
import os
os.environ['BUCKET'] = 'qwiklabs-gcp-00-e0281c0549f5'
os.environ['PROJECT'] = 'qwiklabs-gcp-00-e0281c0549f5'
os.environ['REGION'] = 'us-central1'

In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [ ]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [ ]:
sampling_query = "SELECT COUNT(weight_pounds) AS num_babies FROM (" + query + ") WHERE MOD(ABS(hashmonth), 10) < 8 AND RAND() < 0.0004"
print(sampling_query)

SELECT COUNT(weight_pounds) AS num_babies FROM (
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
) WHERE MOD(ABS(hashmonth), 10) < 8 AND RAND() < 0.0004


In [ ]:
sampling_df = bigquery.Client().query(sampling_query).to_dataframe()
sampling_df.head()

,num_babies
0,10923


In [ ]:
sampling_query = """
SELECT * FROM (
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000 AND weight_pounds IS NOT null AND gEstation_weeks IS NOT null
) 
WHERE MOD(ABS(hashmonth), 10) < 8 AND RAND() < 0.0004
"""

In [ ]:
sampling_df = bigquery.Client().query(sampling_query).to_dataframe()
sampling_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.374463,False,25,1,44,1403073183891835564
1,7.561856,True,26,1,37,7445587375556638376
2,7.251004,False,18,1,40,9068386407968572094
3,6.519069,False,37,1,40,2700219941840496344
4,7.154000,True,26,1,42,6749419160304400104


In [ ]:
sampling_df.shape

(10827, 6)

In [ ]:
sampling_df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,10827.000000,10827.000000,10827.000000,10827.000000,1.082700e+04
mean,7.199704,27.316616,1.037129,38.548074,4.371356e+18
std,1.350902,6.125939,0.200469,2.614843,2.791597e+18
min,0.524700,13.000000,1.000000,17.000000,7.493147e+16
25%,6.558752,22.000000,1.000000,38.000000,1.639186e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.024826,32.000000,1.000000,40.000000,6.888635e+18
max,15.500702,50.000000,5.000000,47.000000,9.183606e+18


In [ ]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,-2126480030009879160,344357
1,8904940584331855459,344191
2,6691862025345277042,338820
3,-1525201076796226340,303664
4,5934265245228309013,324598


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [ ]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) = 3 AND RAND() < 0.0005"
traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13397 examples in the train dataset and 3253 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [ ]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.825512,False,32,1,39.0,3095933535584005890
1,6.393406,True,37,1,39.0,3095933535584005890
2,6.937947,False,21,1,38.0,3095933535584005890
3,7.561856,False,25,1,41.0,3095933535584005890
4,6.124442,True,20,1,40.0,3095933535584005890


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [ ]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13382.000000,13397.000000,13397.000000,13306.000000,1.339700e+04
mean,7.236979,27.294096,1.031276,38.633398,3.544354e+17
std,1.303598,6.099915,0.186090,2.573354,5.209314e+18
min,0.500449,13.000000,1.000000,17.000000,-9.183606e+18
25%,6.563162,22.000000,1.000000,38.000000,-3.340563e+18
50%,7.312733,27.000000,1.000000,39.000000,-2.605984e+17
75%,8.046873,32.000000,1.000000,40.000000,5.896568e+18
max,11.993147,50.000000,4.000000,47.000000,8.599690e+18


In [ ]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [ ]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.825512,False,32,1,39.0,3095933535584005890
1,6.393406,True,37,1,39.0,3095933535584005890
2,6.937947,False,21,1,38.0,3095933535584005890
3,7.561856,False,25,1,41.0,3095933535584005890
4,6.124442,True,20,1,40.0,3095933535584005890


In [ ]:
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.825512,False,32,Single(1),39.0,3095933535584005890
1,6.393406,True,37,Single(1),39.0,3095933535584005890
2,6.937947,False,21,Single(1),38.0,3095933535584005890
3,7.561856,False,25,Single(1),41.0,3095933535584005890
4,6.124442,True,20,Single(1),40.0,3095933535584005890


In [ ]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13392,6.137669,Unknown,30,Single(1),42.0,-774501970389208065
13393,7.813183,Unknown,22,Single(1),40.0,-774501970389208065
13394,6.999677,Unknown,22,Single(1),39.0,-774501970389208065
13395,8.624484,Unknown,26,Single(1),42.0,-774501970389208065
13396,6.124442,Unknown,24,Single(1),40.0,-774501970389208065


In [ ]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26592.000000,26592.000000,26592.000000,2.659200e+04
mean,7.239440,27.298360,38.638688,3.524238e+17
std,1.300336,6.097631,2.555278,5.209030e+18
min,0.500449,13.000000,17.000000,-9.183606e+18
25%,6.563162,22.000000,38.000000,-3.340563e+18
50%,7.312733,27.000000,39.000000,-2.605984e+17
75%,8.047424,32.000000,40.000000,5.896568e+18
max,11.993147,50.000000,47.000000,8.599690e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [ ]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [ ]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   6444 eval.csv
  26606 train.csv
  33050 total
==> eval.csv <==
7.25100379718,False,23,Single(1),39.0,7146494315947640619
7.936641432,True,19,Single(1),39.0,6244544205302024223
6.6248909731,True,27,Single(1),37.0,1891060869255459203
7.31273323054,True,33,Single(1),38.0,2246942437170405963
6.13326012884,True,42,Single(1),36.0,6365946696709051755
7.43839671988,False,23,Single(1),40.0,4740473290291881219
7.936641432,False,23,Single(1),40.0,7146494315947640619
6.52788757782,True,39,Single(1),36.0,4740473290291881219
6.75055446244,False,25,Single(1),34.0,8904940584331855459
6.20821729792,False,33,Single(1),38.0,6365946696709051755

==> train.csv <==
7.87491199864,True,27,Single(1),40.0,774501970389208065
9.31232594688,True,33,Single(1),38.0,774501970389208065
9.37626000286,True,31,Single(1),40.0,774501970389208065
7.3744626639,True,31,Single(1),38.0,774501970389208065
8.5098433132,False,34,Single(1),38.0,774501970389208065
7.3744626639,False,28,Single(1),39.0,774501970389208065
7.18706974

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License